# How Import works ?

In [ ]:
import sys

In [ ]:
# Append the path where we have some modules and packages
sys.path.append('/home/sasidhar/python_programs/how_import_works')

In [ ]:
import importlib

### Import under the hood is just a __import__ function call

In [ ]:
test_module = importlib.__import__('test_module')
test_module

### Module vs Package
Python has only one type of <u>module</u> object. All packages are modules, but not all packaes are modules
Packages are just special kind of modules
Any module that contains \__path\__ attribute is considered as a package

In [ ]:
# Following will raise as module1 is not yet imported
sys.modules['module1']

In [ ]:
# Let's import module1 now

import module1

In [ ]:
# Please note that module1 does not have __path__ attribute
print(dir(module1))

In [ ]:
#Let's now check sys.modules again
m1 = sys.modules['module1']
m1

In [ ]:
m1 is module1

In [ ]:
sys.modules['pkg']

In [ ]:
# Let's import a package now
# We are now importing module bar_module which is in package pkg.bar
# Even though, we import bar_module, it imports all of the necessary parent packages

from pkg.bar import bar_module

In [ ]:
# Package is not in our __main__ namespace
print(dir(pkg))

In [ ]:
# Let's check if pkg is present in sys.modules
pkg = sys.modules['pkg']
pkg

In [ ]:
pkg.bar = sys.modules['pkg.bar']
pkg.bar

In [ ]:
print(dir(pkg))

In [ ]:
# Please note the __path__ attribute on package
pkg.__path__

## Finders ( sys.meta_path)

In [ ]:
# Following function iterates thru each of the finder in sys.meta_path and returs a loader
# if it finds a loader

def get_meta_path_loader(module_name):
    for finder in sys.meta_path:
        loader = finder.find_module(module_name)
        if loader:
            return loader
    else:
        raise ImportError

In [ ]:
loader = get_meta_path_loader('pandas')
loader

In [ ]:
sys.modules['pandas']

In [ ]:
m = loader.load_module()
m

In [ ]:
sys.modules['pandas']

In [ ]:
m1 = get_meta_path_loader('module2')
m1

## Path Finders

In [ ]:
for path in list(sys.path_importer_cache.keys())[0:10]:
    print(path)

In [ ]:
sys.path_hooks

### Using zipimporter to find a package 

In [ ]:
zip_importer_class = sys.path_hooks[0]

zip_importer = zip_importer_class('mypackage.zip')
zip_importer

In [ ]:
mypackage = zip_importer.load_module('mypackage')
mypackage

In [ ]:
sys.modules['mypackage']

In [ ]:
mypackage.subpkg1 = zip_importer.load_module('mypackage/subpkg1')
mypackage.subpkg1

In [ ]:
sp1_module = zip_importer.load_module('mypackage/subpkg1/sp1_module1')

In [ ]:
list(sys.modules.keys())[-5:]

### Using FileFinder to find a package

In [ ]:
file_finder = sys.path_hooks[1]
file_finder

In [ ]:
ff = file_finder('/home/sasidhar/python_programs/how_import_works/module2')
ff

In [ ]:
spam_loader = ff.find_module('spam')
spam_loader

In [ ]:
spam = spam_loader.load_module()
spam

In [ ]:
sys.modules['spam']

## Custom Blocking Importer

In [ ]:
# Following is a custom class with find_module and load_module methods

class ImportBlocker(object):
    def __init__(self, *args):
        self.module_names = args
 
    def find_module(self, fullname, path=None):
        if fullname in self.module_names:
            return self
        return None # this importer is not able to find the module, move to the next in sys.meta_path
 
    def load_module(self, name):
        raise ImportError("%s is blocked and cannot be imported" % name)

In [ ]:
sys.meta_path.insert(0,ImportBlocker('spyder'))

In [ ]:
sys.meta_path

In [ ]:
import spyder

### Namespace Package

### Python 3.6

In [7]:
import importlib
import sys

In [8]:
spec = importlib.util.find_spec('os')
spec

ModuleSpec(name='os', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f8fa79c2dd8>, origin='/home/sasidhar/anaconda36/lib/python3.6/os.py')

In [9]:
os = importlib.util.module_from_spec(spec)

In [10]:
spec.loader.exec_module(os)

In [12]:
sys.modules['os']

<module 'os' from '/home/sasidhar/anaconda36/lib/python3.6/os.py'>